## Tutorial
### Quick start
Create pipeline task with task decorators:

In [ ]:
import dagpipe


@dagpipe.task
def run_a(param):
    return f"Output of A with {param}"

@dagpipe.task
def run_b(a):
    return f"Output of B with {a}"

@dagpipe.task
def run_c(b):
    return f"Output of C with {b}"

Link them:

In [ ]:
a = run_a(param="Initial input for A")
b = run_b(a)
c = run_c(b)

Define pipeline:

In [ ]:
simple_pipeline = dagpipe.Pipeline(
  input=a,
  outputs=[c]
)

Visualize:  


*Note: This step would only works if you will have properly installed Graphviz on your computer.*

In [ ]:
dagpipe.visualize(simple_pipeline)

### Functionality overview

#### Task
Two types of tasks are supported:
1. Function task defined with `task` decorator

In [ ]:
@dagpipe.task
def A(param):
    return f"Output of A with {param}"

@dagpipe.task
def B(a):
    return f"Output of B with {a}"

@dagpipe.task
def C(b):
    return f"Output of C with {b}"

@dagpipe.task
def D(a, c):
    return f"Output of D with {a} and {c}"

2. Method tasks defined with `method_task` decorator

In [ ]:
class ExampleClass:
    @dagpipe.method_task
    def E(self, d):
        return f"Output of E with {d}"

    @dagpipe.method_task
    def F(self, e):
        return f"Output of F with {e}"

    @dagpipe.method_task
    def G(self, inp):
        return f"Output of G with {inp}"

    @dagpipe.method_task
    def __call__(self, inp):
        return f"Output from Exampleclass with {inp}"

Tasks flow is defined in very intuitive way (like normal functions execution).  
Note that:
1. You can mix in single pipeline tasks with methods tasks.
2. You can put more tasks to single task   

In [ ]:
example = ExampleClass()

a = A(param="Initial input for A")
b = B(a)
c = C(b)
d = D(a, c)
e = example.E(d)
f = example.F(e)
g = example.G(c)
ec = example(g)

#### Pipeline
To define pipeline simply pass input and outputs to Pipeline.  
You can define as many output as you want, but only one input.


In [ ]:
pipeline = dagpipe.Pipeline(input=a, outputs=[f, ec])

You can run the same pipeline with different parameters. Your result would be a list with values for each defined output.

In [ ]:
result = pipeline.run()
print(result)

new_result = pipeline.run(param="## New input for A ##")
print(new_result)

new_result2 = pipeline.run(param="@@ New input for A @@")
print(new_result2)


But be aware, that when you run it again without parameters it will use last defined parameter

In [ ]:
result = pipeline.run()
print(result)

`visualize` function create `matplotlib` plot that you can customize.  

Alternatively you can save visualization fo file instead of plotting, by specifying parameter to_file: `dagpipe.visualize(pipeline, to_file="path/to/file")`


Tasks are named with theirs functions names. Method tasks like `ClassName.function_name`, but there is one special case: `__call__` method is named only with class name.
  

In [ ]:
import matplotlib.pyplot as plt


dagpipe.visualize(pipeline)
plt.title("Example pipeline")
plt.gcf().set_size_inches(4, 6)


### Dig deeper 
`dagpipe.task` decorator change function behavior in a way that when it is called it saves base function with its arguments to `Task` object instead of calling it. 

In [ ]:
@dagpipe.task
def foo(x):
  return x

x = 1
print(f"x type before foo processing - {type(x)}")
x = foo(x)
print(f"x type after  foo processing - {type(x)}")

You can check values that are stored by Task x:

In [ ]:
print("Stored function:", x.func)
print("Stored args: \t", x.args)
print("Stored kwargs: \t", x.kwargs)

If you want run stored function, it is possible with `run` method, but normally `Pipeline` object would do it for you.

In [ ]:
x.run()

Task stores information about input arguments that that are provided to it, so when you put another Task as input to your task, it will have information about function that should be run before.  

In [ ]:
@dagpipe.task
def foo2(x):
  return x

inp = 1
x = foo(inp)
x2 = foo2(x)

print(f"input arg for x2 is '{x2.args[0]}' which type is {type(x2.args[0])}")

Basically tuple (*task*, *task argument*) is edge of directional computing graph. `dagpipe.Pipeline` collect information about all edges, and sort them in right execution order. After that you can run whole pipeline. 

In [ ]:
pipeline = dagpipe.Pipeline(input=x, outputs=[x2])
pipeline.run()